In [1]:
# !curl -O https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
# !unzip -q kagglecatsanddogs_3367a.zip
# !ls PetImages

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
image_size = (32, 32)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

Found 23410 files belonging to 2 classes.
Using 18728 files for training.
Found 23410 files belonging to 2 classes.
Using 4682 files for validation.


In [3]:
data_augmentation = keras.Sequential(
    [
        tf.keras.layers.experimental.preprocessing.Rescaling(scale=1/255.),
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)

In [4]:
train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

In [5]:
def make_model(input_shape, num_classes, dual=False):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    x = data_augmentation(inputs)

    # Entry block
    x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(x)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)
        
        if dual:
            x = layers.concatenate([layers.MaxPooling2D(3, strides=2, padding="same")(x),
                                    layers.AveragePooling2D(3, strides=2, padding="same")(x)])
            residual = layers.Conv2D(size*2, 1, strides=2, padding="same")(
            previous_block_activation
        )
        else:
            x = layers.MaxPooling2D(3, strides=2, padding="same")(x)
            residual = layers.Conv2D(size, 1, strides=2, padding="same")(
                previous_block_activation
        )
        
        
        # Project residual
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)




In [6]:
epochs = 50

In [7]:
model = make_model(input_shape=image_size + (3,), num_classes=2, dual=False)
# keras.utils.plot_model(model, show_shapes=False)

callbacks = [
#     keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss=keras.losses.BinaryCrossentropy(label_smoothing=0.1),
    metrics=["accuracy"],
)
model.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)

Epoch 1/50
586/586 [==============================] - 47s 80ms/step - loss: 0.6594 - accuracy: 0.6327 - val_loss: 0.7634 - val_accuracy: 0.4957
Epoch 2/50
586/586 [==============================] - 45s 77ms/step - loss: 0.5987 - accuracy: 0.7053 - val_loss: 0.9052 - val_accuracy: 0.5043
Epoch 3/50
586/586 [==============================] - 45s 77ms/step - loss: 0.5702 - accuracy: 0.7368 - val_loss: 1.8540 - val_accuracy: 0.5043
Epoch 4/50
586/586 [==============================] - 45s 77ms/step - loss: 0.5511 - accuracy: 0.7558 - val_loss: 4.1670 - val_accuracy: 0.4957
Epoch 5/50
586/586 [==============================] - 45s 77ms/step - loss: 0.5455 - accuracy: 0.7605 - val_loss: 1.5176 - val_accuracy: 0.4957
Epoch 6/50
586/586 [==============================] - 45s 77ms/step - loss: 0.5317 - accuracy: 0.7735 - val_loss: 3.3401 - val_accuracy: 0.4957
Epoch 7/50
586/586 [==============================] - 46s 78ms/step - loss: 0.5201 - accuracy: 0.7840 - val_loss: 1.7524 - val_accuracy:

In [8]:
model = make_model(input_shape=image_size + (3,), num_classes=2, dual=False)
# keras.utils.plot_model(model, show_shapes=False)

callbacks = [
#     keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss=keras.losses.BinaryCrossentropy(label_smoothing=0.0),
    metrics=["accuracy"],
)
model.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)

Epoch 1/50
586/586 [==============================] - 49s 84ms/step - loss: 0.6552 - accuracy: 0.6388 - val_loss: 0.6896 - val_accuracy: 0.5265
Epoch 2/50
586/586 [==============================] - 48s 83ms/step - loss: 0.5689 - accuracy: 0.7132 - val_loss: 1.2457 - val_accuracy: 0.4957
Epoch 3/50
586/586 [==============================] - 49s 83ms/step - loss: 0.5341 - accuracy: 0.7367 - val_loss: 0.7581 - val_accuracy: 0.4957
Epoch 4/50
586/586 [==============================] - 48s 83ms/step - loss: 0.5112 - accuracy: 0.7563 - val_loss: 0.7029 - val_accuracy: 0.5019
Epoch 5/50
586/586 [==============================] - 48s 83ms/step - loss: 0.4951 - accuracy: 0.7669 - val_loss: 3.3199 - val_accuracy: 0.4957
Epoch 6/50
586/586 [==============================] - 49s 84ms/step - loss: 0.4835 - accuracy: 0.7727 - val_loss: 0.6880 - val_accuracy: 0.5446
Epoch 7/50
586/586 [==============================] - 49s 83ms/step - loss: 0.4698 - accuracy: 0.7804 - val_loss: 0.8211 - val_accuracy:

In [ ]:
model = make_model(input_shape=image_size + (3,), num_classes=2, dual=False)
# keras.utils.plot_model(model, show_shapes=False)

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"],
)
model.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)

Epoch 1/50
586/586 [==============================] - 50s 85ms/step - loss: 0.6537 - accuracy: 0.6344 - val_loss: 0.6919 - val_accuracy: 0.5276
Epoch 2/50
586/586 [==============================] - 49s 84ms/step - loss: 0.5661 - accuracy: 0.7113 - val_loss: 3.8972 - val_accuracy: 0.4957
Epoch 3/50
586/586 [==============================] - 48s 82ms/step - loss: 0.5334 - accuracy: 0.7396 - val_loss: 2.6126 - val_accuracy: 0.4957
Epoch 4/50
586/586 [==============================] - 49s 84ms/step - loss: 0.5105 - accuracy: 0.7533 - val_loss: 1.1532 - val_accuracy: 0.4957
Epoch 5/50
586/586 [==============================] - 48s 83ms/step - loss: 0.4924 - accuracy: 0.7635 - val_loss: 6.7288 - val_accuracy: 0.4957
Epoch 6/50
586/586 [==============================] - 48s 82ms/step - loss: 0.4758 - accuracy: 0.7780 - val_loss: 1.7603 - val_accuracy: 0.4957
Epoch 7/50
586/586 [==============================] - 49s 83ms/step - loss: 0.4621 - accuracy: 0.7880 - val_loss: 1.5380 - val_accuracy:

In [10]:
model = make_model(input_shape=image_size + (3,), num_classes=2, dual=False)
# keras.utils.plot_model(model, show_shapes=False)

callbacks = [
    keras.callbacks.ModelCheckpoint("ls01_save_at_{epoch}.h5"),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss=keras.losses.BinaryCrossentropy(label_smoothing=0.01),
    metrics=["accuracy"],
)
model.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)

586/586 [==============================] - 49s 83ms/step - loss: 0.6486 - accuracy: 0.6382 - val_loss: 0.7411 - val_accuracy: 0.5043
Epoch 2/50
586/586 [==============================] - 48s 82ms/step - loss: 0.5755 - accuracy: 0.7074 - val_loss: 0.7234 - val_accuracy: 0.5064
Epoch 3/50
586/586 [==============================] - 48s 81ms/step - loss: 0.5364 - accuracy: 0.7377 - val_loss: 2.5416 - val_accuracy: 0.4957
Epoch 4/50
586/586 [==============================] - 48s 81ms/step - loss: 0.5191 - accuracy: 0.7506 - val_loss: 0.7117 - val_accuracy: 0.5126
Epoch 5/50
586/586 [==============================] - 48s 82ms/step - loss: 0.4999 - accuracy: 0.7679 - val_loss: 1.0414 - val_accuracy: 0.4962
Epoch 6/50
586/586 [==============================] - 48s 83ms/step - loss: 0.4851 - accuracy: 0.7760 - val_loss: 2.8568 - val_accuracy: 0.4957
Epoch 7/50
586/586 [==============================] - 48s 82ms/step - loss: 0.4730 - accuracy: 0.7827 - val_loss: 3.4902 - val_accuracy: 0.4957
Epo

In [12]:
def name(var):
    match = [k for k, v in globals().items() if v == var]
    return match[0].replace('_history', '') if match else None

def plot(metric, *history):
    
    getter = metric.replace('train_', '')
    
    for h in history:
        plt.plot(h.history['{}'.format(getter)])
    plt.title('{}, batch size: 32'.format(metric).upper())
    plt.ylabel('{}'.format(metric))
    plt.xlabel('epoch')
    plt.legend(list(map(name, history)), loc='upper left')
    plt.show()

In [13]:
for metric in ['train_loss', 'val_loss', 'train_accuracy', 'val_accuracy']:
    plot(metric, baseline_history, dual_history)

NameError: name 'baseline_history' is not defined